# Rendering Hello Box2D

First, let's import the required libraries.

In [1]:
import math
import numpy as np

import gymnasium as gym
from gymnasium.spaces import Discrete,MultiDiscrete

import Box2D
from Box2D.b2 import fixtureDef, polygonShape

import pygame

Basic variables definition 

In [5]:
timeStep = 1/60 #Time step is set at 60 Hertz (1/60 seconds)
velocityIterations = 6 #Max iterations to correctly infer velocity from physics laws at each time st
positionIterations = 2 #Max iterations to correctly infer position from physics laws at each time step
FPS = 50  # Frames per second
WINDOW_W = 1000 # width of the pygame window
WINDOW_H = WINDOW_W # height of the pygame window
SIZE = 20 # size of the square coordinates in the pygame window
pix_square_size = WINDOW_H/SIZE # total number of square

g = (0,-10) # gravity vector
p = (0, 40) # initial center of mass of rocket
p_static = (0,-9) # center of mass of earth
vertx_p = (1,1) # distance from rocket center of mass to its edges
vertx_pst = (50,10) # think of 10 as the radius of earth

Main class definition

In [6]:
class HelloBox2D(gym.Env):

    ####################################################################################################
    # RENDER MODES
    # different modes to render the landing
    ####################################################################################################

    metadata = {
        "render_modes": [
            "human", #continuous rendering
            "rgb_array", #single frame representing current state
            "state_pixels", #number of pixels defined by user -> to be used for tensorboard
        ],
        "render_fps": FPS, #frame per seconds
    }

    ####################################################################################################
    # INITIALISATION
    ####################################################################################################

    def __init__(
        self,
        render_mode = None
    ):
        
        # Definition of World and other attributes
        self.world = Box2D.b2World(g)

        # Definition of some attributes needed for rendering
        self.screen = None
        self.clock = None
        self.surf = None
        
        # Definition of Static Body (earth, can also be round!)
        self.GroundBody : Box2D.b2Body = self.world.CreateBody(
            position = p_static,
            angle = 0.0,
            fixtures = fixtureDef(
                shape = polygonShape(box = vertx_pst)
            )
        )

        # Definition of Dynamic Body (rocket)
        self.position = p
        self.body : Box2D.b2Body = self.world.CreateDynamicBody(
            position=self.position,
            angle = 0.0,
            fixtures = fixtureDef(
                shape = polygonShape(box = vertx_p),
                density = 1,
                friction = 0.3
            )
        )

        # No action is available, dynamic is based on gravitational force only
        self.action_space = {}

        # the observation will be the coordinates of the Static Body
        self.observation_space = {}

        # Definition of render mode
        self.render_mode = render_mode

    ####################################################################################################
    # STEP
    ####################################################################################################

    def step(self, action = None):
        #current position
        s = self.body.position.y 

        #execute step
        self.world.Step(timeStep, velocityIterations, positionIterations) 
        self.state = self.body.position #update state
        step_reward = 0 #update reward
        terminated = False
        truncated = False
        
        #when rocket lands, terminate
        if (s-self.body.position.y) < 0.000000001:
            terminated = True 
            print('Landing achieved, current altitute = ', self.state.y)

        #call render function and show animation
        if self.render_mode == "human":
            self.render() 
        return self.state, step_reward, terminated, truncated, {}

    ####################################################################################################
    # RESET
    ####################################################################################################

    def reset(self, seed = None, options = None):
        super().reset(seed=seed)
        self._destroy()
        
        self.body.position = p

        if self.render_mode == "human":
            self.render()
        return self.step(None)[0], {}

    ####################################################################################################
    # DESTROY
    ####################################################################################################

    def _destroy(self):
        self.world.DestroyBody(self.body)
        self.world.DestroyBody(self.GroundBody)


    ####################################################################################################
    # RENDER 
    # (see https://medium.com/data-science-in-your-pocket/game-development-using-pygame-reinforcement-learning-with-example-f5b78c768610
    #  https://www.gymlibrary.dev/content/environment_creation/, https://stackoverflow.com/questions/19780411/pygame-drawing-a-rectangle
    #  and also the pygame website: https://www.pygame.org/docs/tut/PygameIntro.html)
    ####################################################################################################

    def render(self):
        return self._render(self.render_mode)
  
    def _render(self, mode: str):
        assert mode in self.metadata["render_modes"]

        # Activate pygame window
        if self.screen is None and mode == "human":
            self.screen = pygame.display.set_mode((WINDOW_W, WINDOW_H)) 

        if self.clock is None:
            self.clock = pygame.time.Clock() # define the environment clock 

        self.surf = pygame.Surface((WINDOW_W, WINDOW_H)) #This is a reference to the window

        self.surf.fill((255,255,255))        
        
        # Add the bodies 
        obj = self.body
        trans = obj.transform #Vector with current center of mass of the body
        lefttop = trans * vertx_p #The current position of the top left vertex of the body in the original grid (not pygame) coordinates
        pygame.draw.rect(self.surf,(0,0,250) ,(400-lefttop[0],400-lefttop[1],25,50))
        
        obj = self.GroundBody
        trans = obj.transform 
        lefttop = trans * vertx_pst
        pygame.draw.rect(self.surf,(0,255,0),(0,400+50-lefttop[1],5000,5000))
        
        # Add the gridline
        for x in range(SIZE + 1):
            pygame.draw.line(
                self.surf,
                0,
                (0, pix_square_size * x),
                (WINDOW_H, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                self.surf,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, WINDOW_H),
                width=3,
            )

        if mode == "human":
            assert self.screen is not None 
            self.clock.tick(self.metadata["render_fps"]) #This ensures that human-rendering occurs at the predefined framerate (might be as fast as the output code otherwise!).
            self.screen.blit(self.surf,(0,0)) #Thiscopies the drawings from surf to the pygame window
            pygame.display.flip() #This will update the content of the entire display (see https://stackoverflow.com/questions/29314987/difference-between-pygame-display-update-and-pygame-display-flip)

    ####################################################################################################
    # CLOSE
    # close the pygame window
    ####################################################################################################    
    def close(self):
        if self.screen is not None:
            pygame.display.quit()
            self.isopen = False
            pygame.quit()
        

In [11]:
env = HelloBox2D(render_mode="human")    

quit = False
while not quit:
    total_reward = 0.0
    steps = 0
    restart = False
    while True:
        s, r, terminated, truncated, info = env.step()
        steps += 1
        if terminated or truncated or restart or quit:
            quit = True
            break
        print(f"step = {steps}, altitude = {s.y:+0.2f}")
env.close()

step = 1, altitude = +40.00
step = 2, altitude = +39.99
step = 3, altitude = +39.98
step = 4, altitude = +39.97
step = 5, altitude = +39.96
step = 6, altitude = +39.94
step = 7, altitude = +39.92
step = 8, altitude = +39.90
step = 9, altitude = +39.88
step = 10, altitude = +39.85
step = 11, altitude = +39.82
step = 12, altitude = +39.78
step = 13, altitude = +39.75
step = 14, altitude = +39.71
step = 15, altitude = +39.67
step = 16, altitude = +39.62
step = 17, altitude = +39.58
step = 18, altitude = +39.53
step = 19, altitude = +39.47
step = 20, altitude = +39.42
step = 21, altitude = +39.36
step = 22, altitude = +39.30
step = 23, altitude = +39.23
step = 24, altitude = +39.17
step = 25, altitude = +39.10
step = 26, altitude = +39.03
step = 27, altitude = +38.95
step = 28, altitude = +38.87
step = 29, altitude = +38.79
step = 30, altitude = +38.71
step = 31, altitude = +38.62
step = 32, altitude = +38.53
step = 33, altitude = +38.44
step = 34, altitude = +38.35
step = 35, altitude = +